In [7]:
# !pip install groq

In [8]:
# !pip install langchain langchain-community langchain-groq

In [9]:
# !pip install faiss-cpu

In [10]:
from groq import Groq
import os
from dotenv import load_dotenv
load_dotenv()

client = Groq(api_key=os.getenv("GROQ_API_KEY"))

In [11]:
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# loading documents to be used for RAG 
text_folder = "rag_files"  

all_documents = []
for filename in os.listdir(text_folder):
    if filename.lower().endswith(".txt"):
        file_path = os.path.join(text_folder, filename)
        loader = TextLoader(file_path)
        all_documents.extend(loader.load())

# generate embeddings
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")

# create vector database w FAISS 
vector_store = FAISS.from_documents(all_documents, embeddings)
retriever = vector_store.as_retriever()


In [ ]:
def main():
    print("Welcome to the RAG Assistant. Type 'exit' to quit.\n")
    
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() == "exit":
            print("Exiting…")
            break

        # get relevant documents
        relevant_docs = retriever.get_relevant_documents(user_input)
        retrieved_context = "\n\n".join([doc.page_content for doc in relevant_docs])

        # system prompt
        system_prompt = (
            "You are a helpful assistant. "
            "Use ONLY the following knowledge base context to answer the user. "
            "If the answer is not in the context, say you don't know.\n\n"
            f"Context:\n{retrieved_context}"
        )

        # messages for LLM 
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ]

        # generate response
        response = client.chat.completions.create(messages = messages, model="llama-3.3-70b-versatile")
        assistant_message = response.choices[0].message.content
        print(f"\nAssistant: {assistant_message}\n")


In [15]:
main()

Welcome to the RAG Assistant. Type 'exit' to quit.


Assistant: Hello. How can I assist you with the Code of Conduct at TechNova Solutions?


Assistant: The Code of Conduct for TechNova Solutions consists of the following sections:

1. **Professionalism**: 
   - All employees must maintain respectful and professional behavior.
   - Harassment or discrimination will not be tolerated.

2. **Confidentiality**: 
   - Employees must not disclose client or internal data to unauthorized parties.

3. **Use of Company Assets**: 
   - Laptops and software licenses must be used only for work purposes.

4. **Social Media**: 
   - Employees should avoid discussing confidential or controversial company topics online.

5. **Disciplinary Action**: 
   - Violation of the code can lead to warnings, suspension, or termination based on severity.

Exiting…
